In [9]:
import openai
import os
from IPython.display import Markdown, HTML, display
import json
import panel as pn
import ipywidgets as widgets

## Setup

In [2]:
from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv())

openai.api_key = os.getenv('OPENAI_API_KEY')

### Helper Function

In [3]:
def get_completion_message(messages, model="gpt-4o-mini", temperature=0):
    response = openai.chat.completions.create(
        model=model,
        messages=messages,
        temperature= temperature,
    )
    return response.choices[0].message.content

# The Chat Format

In this notebook, you will explore how you can utilize the chat format to have extended conversations with chatbots personalized or specialized for specific tasks or behaviors.

In [4]:
messages =  [  
{'role':'system', 'content':'You are an assistant that speaks like Gen Z.'},    
{'role':'user', 'content':'tell me a joke'},   
{'role':'assistant', 'content':'Why did the chicken cross the road'},   
{'role':'user', 'content':'I don\'t know'}  ]

In [5]:
response = get_completion_message(messages, temperature=1)
print(response)

To get to the other side! Classic, right? 😂 But for real, I can hit you with something fresher if you want!


In [6]:
messages =  [  
{'role':'system', 'content':'You are friendly chatbot.'},
{'role':'user', 'content':'Hi, my name is Albatross'},
{'role':'assistant', 'content': "Hi Albatross! It's nice to meet you. \
Is there anything I can help you with today?"},
{'role':'user', 'content':'Yes, you can remind me, What is my name?'}  ]
response = get_completion_message(messages, temperature=1)
print(response)

Your name is Albatross! Is there anything else you'd like to know or discuss?


## OrderBot

In [7]:
def collect_messages(_):
    prompt = inp.value_input
    inp.value = ''
    context.append({'role':'user', 'content':f"{prompt}"})
    response = get_completion_message(context) 
    context.append({'role':'assistant', 'content':f"{response}"})
    panels.append(
        pn.Row('User:', pn.pane.Markdown(prompt, width=600)))
    panels.append(
        pn.Row('Assistant:', pn.pane.Markdown(response, width=600, styles={'background-color': '#F6F6F6'})))
 
    return pn.Column(*panels)


In [12]:
context = [{
    'role': 'system',
  'content': """
You are OrderBot, an automated service to collect orders for a pizza restaurant. 
You first greet the customer, then collect the order, 
and then ask if it's a pickup or delivery. 
You wait to collect the entire order, then summarize it and check for a final 
time if the customer wants to add anything else. 
If it's a delivery, you ask for an address. 
Finally you collect the payment.
Make sure to clarify all options, extras and sizes to uniquely 
identify the item from the menu.
You respond in a short, very conversational friendly style. 
The menu includes 
pepperoni pizza  12.95, 10.00, 7.00 
cheese pizza   10.95, 9.25, 6.50 
eggplant pizza   11.95, 9.75, 6.75 
fries 4.50, 3.50 
greek salad 7.25 
Toppings: 
extra cheese 2.00, 
mushrooms 1.50 
sausage 3.00 
canadian bacon 3.50 
AI sauce 1.50 
peppers 1.00 
Drinks: 
coke 3.00, 2.00, 1.00 
sprite 3.00, 2.00, 1.00 
bottled water 5.00 
"""
}]
 
# Widgets (unchanged)
input_box = widgets.Textarea(placeholder="Type your message here...")
send_button = widgets.Button(description="Chat")
output_box = widgets.Output(layout={'border': '1px solid black', 'height': '400px', 'overflow': 'auto'})

# Chat completion helper (uses the shared `context`)
def get_completion_message(messages, model="gpt-4o-mini", temperature=0):
    response = openai.chat.completions.create(
        model=model,
        messages=messages,
        temperature=temperature,
    )
    return response.choices[0].message.content

# Callback (append to `context` on every turn)
def send_message(b):
    user_message = input_box.value.strip()
    if not user_message:
        return

    # Show user bubble
    with output_box:
        display(HTML(
            f"<div style='text-align:right; margin:4px;'>"
            f"<span style='background-color:#d1e7ff; padding:8px; border-radius:8px; display:inline-block; max-width:70%;'>"
            f"<b>You:</b> {user_message}</span></div>"
        ))

    # Clear input
    input_box.value = ""

    # Add user message into context
    context.append({"role": "user", "content": user_message})

    # Get bot reply with current context (system + full history)
    bot_reply = get_completion_message(context)

    # Add assistant reply into context
    context.append({"role": "assistant", "content": bot_reply})

    # Show assistant bubble
    with output_box:
        display(HTML(
            f"<div style='text-align:left; margin:4px;'>"
            f"<span style='background-color:#f0f0f0; padding:8px; border-radius:8px; display:inline-block; max-width:70%;'>"
            f"<b>Samay:</b> {bot_reply}</span></div>"
        ))

send_button.on_click(send_message)

# Display UI
display(input_box, send_button, output_box)

Textarea(value='', placeholder='Type your message here...')

Button(description='Chat', style=ButtonStyle())

Output(layout=Layout(border_bottom='1px solid black', border_left='1px solid black', border_right='1px solid b…